In [1]:
import pandas as pd, numpy as np, os, re, operator, string, csv
from operator import itemgetter
from collections import Counter, defaultdict

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#.download('words')
words = set(nltk.corpus.words.words())


from gensim import corpora, models, similarities, utils
from gensim.models.tfidfmodel import TfidfModel

unable to import 'smart_open.gcs', disabling that module


In [2]:
os.chdir('C:/Users/elcot/Downloads/New folder')

In [3]:
path = "C:/Users/elcot/Downloads/New folder"

In [4]:
path1="C:/Users/elcot/Downloads/New folder"

In [5]:
import os, glob
import pandas as pd

In [6]:
all_files = glob.glob(os.path.join(path, "df3*.csv"))

In [7]:
all_csv = (pd.read_csv(f, sep=',') for f in all_files)

In [8]:
df_merged   = pd.concat(all_csv, ignore_index=True)

C:\Users\elcot\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [9]:
df_merged.to_csv( "df3.csv")

In [10]:
df=pd.read_csv('df3.csv')

In [11]:
df.head(10)

,Unnamed: 0,Q_and_A,Unnamed: 0.1,answer,keywords,question
0,0,Question:Is this cover the one that fits the o...,0.0,Yes this fits both the nook color and the same...,cover fit nook color shaped tablet,Is this cover the one that fits the old nook c...
1,1,Question:Does it fit Nook GlowLight?\nAnswer ...,1.0,No. The nook color or color tablet,fit nook glowlight color tablet,Does it fit Nook GlowLight?
2,2,Question:Would it fit Nook 1st Edition? 4.9in ...,2.0,I don't think so. The nook color is 5 x 8 so n...,fit nook edition color smaller stay locked close,Would it fit Nook 1st Edition? 4.9in x 7.7in ?
3,3,Question:Will this fit a Nook Color that's 5 x...,3.0,yes,fit nook color,Will this fit a Nook Color that's 5 x 8?
4,4,Question:will this fit the Samsung Galaxy Tab ...,4.0,"No, the tab is smaller than the 'color'",fit samsung galaxy tab nook smaller color,will this fit the Samsung Galaxy Tab 4 Nook 10.1
5,5,Question:does it have a flip stand?\nAnswer :...,5.0,"No, there is not a flip stand. It has a pocket...",flip stand pocket flap nice cover,does it have a flip stand?
6,6,Question:does this have a flip stand\nAnswer ...,6.0,"Hi, no it doesn't",flip stand,does this have a flip stand
7,7,Question:also fits the HD+?\nAnswer :It shoul...,7.0,It should. They are the same size and the char...,fit hd size charging port place,also fits the HD+?
8,8,Question:Does it have 2 positions for the read...,8.0,Yes,position reader horizontal vertical kwod,Does it have 2 positions for the reader? Horiz...
9,9,"Question:Is there a closure mechanism? Bands, ...",9.0,No- it is more like a normal book would be. It...,closure mechanism band magnetic normal book fl...,"Is there a closure mechanism? Bands, magnetic,..."


In [67]:
df.shape

(314091, 5)

In [68]:
df.reset_index

<bound method DataFrame.reset_index of         Unnamed: 0                                           question  \
0                0  Is this cover the one that fits the old nook c...   
1                1                        Does it fit Nook GlowLight?   
2                2     Would it fit Nook 1st Edition? 4.9in x 7.7in ?   
3                3           Will this fit a Nook Color that's 5 x 8?   
4                4   will this fit the Samsung Galaxy Tab 4 Nook 10.1   
5                5                         does it have a flip stand?   
6                6                        does this have a flip stand   
7                7                                 also fits the HD+?   
8                8  Does it have 2 positions for the reader? Horiz...   
9                9  Is there a closure mechanism? Bands, magnetic,...   
10              10                   how far out does the arm extend?   
11              11  can you mount a sound bar using the external "...   
12          

In [69]:
df.isnull().sum()

Unnamed: 0    0
question      0
answer        0
keywords      0
Q_and_A       0
dtype: int64

In [70]:
df = df[df['keywords'].notna()]

In [71]:
docs = df['keywords'].tolist()

In [81]:
processed_docs = [word_tokenize(doc.lower()) for doc in docs] #tokenize and lowercase to generate bag of words

In [82]:
dictionary = corpora.Dictionary(processed_docs)

In [83]:
dictionary.save(path+'dim_items_terms.dict')

In [84]:
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [85]:
corpora.MmCorpus.serialize(path+'dim_items_terms.mm', corpus)

In [86]:
tfidf = TfidfModel(corpus)

In [87]:
tfidfmodelsave=tfidf.save(path+'dim_items_terms.tfidf')

In [88]:
tfidf_corpus=tfidf[corpus]

In [89]:
sorted_tfidf_weights = sorted(tfidf[corpus[0]], key=lambda w: w[1], reverse=True)
for term_id, weight in sorted_tfidf_weights[:5]:
    print(dictionary.get(term_id), weight)

shaped 0.5856416780673531
nook 0.5218843240682374
color 0.37567132399723385
cover 0.3372062336880242
tablet 0.2995695872768057


In [90]:
sims = similarities.Similarity('path1',tfidf[corpus], num_features=len(dictionary))

In [91]:
def keywords_recommendation(keyword_rec,Top_N_Q_and_A):
    if os.path.exists("recommandation.csv"):
        os.remove("recommandation.csv")
    keyword_rec=keyword_rec
    keywords_rec = re.sub(r"http\S+", "", keyword_rec) 
    keywords_rec = keywords_rec.replace('.',' ') 
    keywords_rec=re.sub("<!--?.*?-->","",keywords_rec)
    keywords_rec=re.sub("(\\d|\\W)+"," ",keywords_rec)
    keywords_rec = re.sub(r'\s+',' ',re.sub(r'[^\w \s]','',keywords_rec) ).lower()
    keywords_rec = word_tokenize(keywords_rec) # tokenize for generating bag of words
    keywords_rec= [w for w in keywords_rec if w not in set(stopwords.words('english'))]
    
    # After cleaning the given question keyword using text nlp, if keyword returns empty then given keywords will be assigned/used directly for analysis 
    if len(keywords_rec)==0:
        keywords_rec=word_tokenize(keyword_rec)
        
    # creating model for the keyword 
    query_doc_bow = dictionary.doc2bow(keywords_rec) # get a bag of words from the query_doc
    query_doc_tfidf = tfidf[query_doc_bow] #convert the regular bag of words model to a tf-idf model of the keywords and it's tf-idf value for the question and answer
    
    # Finding similarities between data and the keywords and converting the array into series for sorting    
    
    similarity_array = sims[query_doc_tfidf] # get the array of similarity values between our new keywords and every other keywords. 
    similarity_series = pd.Series(similarity_array.tolist(), index=df.Q_and_A.values) #Convert to a Series
    Top_Q_and_A = similarity_series.sort_values(ascending=False)[:Top_N_Q_and_A] #get the top matching results, 
    
    #Storing the sorted values for further processing.----Print
    Top_Q_and_A.to_csv('recommandation.csv', mode='a',  header=False)
    
    # labeling and values and printing the sorted values as per requirements    
    col_Names=["Q_and_A", "Score"]
    df1 = pd.read_csv("recommandation.csv",names=col_Names)
    df1=df1.sort_values(by=['Score'], ascending=False)
    
    print('==============================')  
    print('keyword: ', keyword_rec )
    print('--------&-------------')  
    print('Most Related Top ', Top_N_Q_and_A , ' Questions and Answers are as:')
    print('==============================')  
    for j in range(Top_N_Q_and_A):
        similarity_score=df1['Score'].loc[j]
        #print('similarity score is ', similarity_score)
        split_lines=''
        split_lines=df1['Q_and_A'].loc[j] 
        print(split_lines.split('\n', 1)[0])
        print(split_lines.split('\n', 1)[1])
        #similarity_score=df1['Score'].loc[j]
        print('similarity score is :', similarity_score)
        print('**********************')    

In [92]:
key_rec=keywords_recommendation('Ram',5)

keyword:  Ram
--------&-------------
Most Related Top  5  Questions and Answers are as:
Question:How much ram?
Answer  :4
similarity score is : 1.0
**********************
Question:Is this the RAM-B-101U
Answer  :Yes it is.
similarity score is : 1.0
**********************
Question:How much RAM does it have?
Answer  :512M
similarity score is : 1.0
**********************
Question:how much ram does it have??
Answer  :512M
similarity score is : 1.0
**********************
Question:Will this work for a 2014 ram 1500?
Answer  :It should.
similarity score is : 0.9595279693603516
**********************
